In [3]:
!pip install spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.0 MB/s eta 0:00:00a 0:00:01


In [4]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime

In [5]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

# Testing

In [110]:
import pdfplumber

def extract_words_with_formatting(page):
    """
    Extracts words along with their formatting details such as font size and font name.

    Args:
        page (pdfplumber.Page): A single page from the PDF.

    Returns:
        List[Dict]: A list of dictionaries containing words and their formatting details.
    """
    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/reports/20220209_JP_Morgan_PEP_PepsiCo-_4Q_Preview-_Expect_Continued_Sales_Momentum_and.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )


Page 1:
Word: 'www.jpmorganmarkets.com', Font: EAXOCN+Arial,Bold, Size: 10.080302399999997, Position: (458.88, 767.0167217088 - 592.6491486585599, 777.0970241088)
Word: 'North', Font: EAXOCN+Arial,Bold, Size: 9.12018240000009, Position: (429.36, 22.573296268799936 - 453.45655733232, 31.693478668800026)
Word: 'America', Font: EAXOCN+Arial,Bold, Size: 9.12018240000009, Position: (455.90131498488, 22.573296268799936 - 491.56369921680005, 31.693478668800026)
Word: 'Equity', Font: EAXOCN+Arial,Bold, Size: 9.12018240000009, Position: (494.16, 22.573296268799936 - 521.71883945712, 31.693478668800026)
Word: 'Research', Font: EAXOCN+Arial,Bold, Size: 9.12018240000009, Position: (524.14563093168, 22.573296268799936 - 564.76340645472, 31.693478668800026)
Word: '04', Font: EAXOCN+Arial, Size: 7.92023759999995, Position: (429.36, 33.838852771200095 - 438.18655657344, 41.759090371200045)
Word: 'February', Font: EAXOCN+Arial, Size: 7.92023759999995, Position: (440.4560782848, 33.838852771200095 - 472

In [56]:
"""provider = "Gilford Securities Inc"

provider_patterns = patterns[provider]

word = "Arial-BoldMT"
size = 10.02

for font_pattern in provider_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word) and word == font_size:
                is_font_matched = True
                #break

print(is_font_matched)"""

TypeError: string indices must be integers, not 'str'

In [219]:
# Test new Structure

patterns = {
    "BGC Partners": {
        "price_patterns": {
            "primary": r"Price Target \(\$\) (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(\w+,",
        },
        "ending_pattern": r"Disclosures Appendix",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Needham": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
            "secondary": r"PRICE TARGET: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"Rating (\w+)",
        },
        "ending_pattern": r"Analyst Certification",
        "font_patterns": [
            {"font_type": r"E[A-Z]+[+]Cambria(-Bold)?", "font_size": 10.0},
        ]
    },
    "BTIG": {
        "price_patterns": {
            "primary": r"\$(\d+(\.\d+)?) 12 month target ",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_pattern": r"Appendix: Analyst Certification and Other Important Disclosures",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?", "font_size": 9.96},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Wells Fargo": {
        "price_patterns": {
            "primary": r"\/Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target\/Prior: \$(\d+(\.\d+)?)",
            "tertiary": r"\/\$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)/\$",
            "secondary": r"Rating (\w+)",
        },
        "ending_pattern": r"Required Disclosures",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]WellsFargoSans(-Light)?(,-SemiBold)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01}
        ]
    },
    "Barclays": {
        "price_patterns": {
            "primary": r"Price Target USD (\d+(\.\d+)?)",
            "secondary": r"Price Target: USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Stock Rating ([A-Za-z]+)",
        },
        "ending_pattern": r"ANALYST(S) CERTIFICATION(S)",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?(Regular,Bold)?", "font_size": 9.0},
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01},
        ]
    },
    "JP Morgan": {
        "price_patterns": {
            "primary": r"Price Target \([A-Za-z0-9\-]+\): \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_pattern": r"Analyst Certification",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?", "font_size": 9.60},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Brean Capital LLC": {
        "price_patterns": {
            "primary": r"PT: \$ (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) PT:\$",
        },
        "ending_pattern": r"Analyst Certification",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Hilliard Lyons": {
        "price_patterns": {
            "primary": r"Price Target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"-- ([A-Za-z]+) --",
        },
        "ending_pattern": r"Analyst Certification",
        "font_patterns": []
    },
    "Alliance Global Partners": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(Ticker:",
        },
        "ending_pattern": r"Imporant Research Disclosures",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Mizuho Securities": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
        },
        "ending_pattern": r"IMPORTANT DISCLOSURES",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Gilford Securities Inc": {
        "price_patterns": {
            "primary": r"\, \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rated: ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_pattern": r"ANALYST CERTIFICATION",
        "font_patterns": [
            {"font_type": r"ArialMT(-BoldMT)?", "font_size": 10.02},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Deutsche Bank": {
        "price_patterns": {
            "primary": r"Price Target \(USD\) (\d+(\.\d+)?)",
            "secondary": r"Price target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
            "secondary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_pattern": r"Appendix 1",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]UniversDeutscheBank-Regular", "font_size": 9.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Pivotal Research Group": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"RATING: ([A-Za-z]+)",
        },
        "ending_pattern": r"Appendix: Important Disclosures",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Helvetica(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Arial", "font_size": 8.04},
        ]
    },
    "Spartan Capital Securities LLC": {
        "price_patterns": {
            "primary": r"T \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)",
        },
        "ending_pattern": r"Important Disclosures",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Cascend Securities -Historical-": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating: ([A-Za-z]+)",
        },
        "ending_pattern": r"Disclosures: ",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 12.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Phillip Securities": {
        "price_patterns": {
            "primary": r"TARGET PRICE USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_pattern": r"Contact Information",
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 10.0},
        ]
    },
    "FinTrust Investment Advisors": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Fintrust Rating: ([A-Za-z]+)",
        },
        "ending_pattern": r"Important Disclosures:",
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "IBI Investment House": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Recommendation: ([A-Za-z]+)",
        },
        "ending_pattern": r"Disclosures",
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    }
}



In [220]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs

def extract_text_with_format(page, provider):
    """
    Extract paragraphs with specific formatting, filter by validity, and extract rating/price.

    Args:
        page (pdfplumber.Page): The page object from pdfplumber.
        provider (str): The name of the provider to extract patterns.

    Returns:
        tuple: Filtered paragraphs, stop extraction flag, extracted rating, and price.
    """
    provider_patterns = patterns[provider]

    # Access patterns
    price_patterns = provider_patterns["price_patterns"]
    rating_patterns = provider_patterns["rating_patterns"]
    ending_pattern = provider_patterns["ending_pattern"]
    font_patterns = provider_patterns["font_patterns"]

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Round text sizes to 3 decimal places
    for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = []
    current_paragraph = []
    current_top = None

    rating = None
    price = None

    # Lookahead buffer for multi-word patterns
    lookahead_buffer = []

    for word in words:
        # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        #print(buffer_text)

        # Extract rating
        if not rating:
            for pattern_key in ["primary", "secondary","tertiary"]:
                pattern = rating_patterns.get(pattern_key)
                if pattern:
                    rating_match = re.search(pattern, buffer_text)
                    if rating_match:
                        rating = rating_match.group(1)
                        break

        # Extract price
        if not price:
            for pattern_key in ["primary", "secondary"]:
                pattern = price_patterns.get(pattern_key)
                if pattern:
                    price_match = re.search(pattern, buffer_text)
                    if price_match:
                        price = price_match.group(1)
                        break

        # Check for ending pattern
        if re.search(ending_pattern, buffer_text):
            return filter_valid_paragraphs(paragraphs), True, rating, price

        # Match word against font patterns
        is_font_matched = False
        for font_pattern in font_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word["fontname"]) and word["size"] == font_size:
                is_font_matched = True
                #break

        if not is_font_matched:
            continue

        # Group words into paragraphs
        if current_top is None or abs(word["top"] - current_top) < 13:  # Adjust threshold as needed
            current_paragraph.append(word["text"])
        else:
            # New paragraph starts
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]

        # Update the current top position
        current_top = word["top"]

    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    return filter_valid_paragraphs(paragraphs), False, rating, price


# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    """
    Extract metadata (date, provider, ticker) from the filename using ticker_map.
    """
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None


In [221]:

# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    """Extract text from all PDFs in a specified directory and store in a DataFrame with unique ID."""
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

            
            # Extract metadata from filename
            date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)

            all_paragraphs = []  
            first_rating = None
            first_price = None  
            stop_extraction = False

            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    if stop_extraction:
                        break  # Exit the loop if stop_extraction is set
            
                    # Extract data from the current page
                    paragraphs, stop_extraction, rating, price = extract_text_with_format(page, provider=provider)
                    
                    # Append paragraphs from the current page
                    all_paragraphs.extend(paragraphs)

                    # Capture the first non-None rating and price
                    if rating is not None and first_rating is None:
                        first_rating = rating
                    if price is not None and first_price is None:
                        first_price = price
                        
            # Add extracted data to the list
            data.append({
                "ID": id_counter,  # Unique ID
                "filename": filename,
                "date": date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "paragraphs": all_paragraphs,
                "target_price":first_price,
                "rating":first_rating
                })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "../preprocessing/reports"  # Replace with your actual folder path

# Run the function and store results in a DataFrame
df_reports = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

# Display the resulting DataFrame to confirm
df_reports.head(15)

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-10-01,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,[BOTTOM LINE: We and almost every other invest...,"4,500",BUY
1,2,20190730_FinTrust_Investment_Advisors_AMZN_Fin...,2019-07-30,FinTrust Investment Advisors,AMZN,Amazon.com Inc.,Consumer Discretionary,[We are issuing this special report due to the...,"1,611",HOLD
2,3,20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...,2022-04-27,Wells Fargo,AAPL,Apple Inc.,Technology,"[Heading into Apple's F2Q22 earnings on 4/28, ...",205.00,Overweight
3,4,20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf,2015-07-28,BGC Partners,AMZN,Amazon.com Inc.,Consumer Discretionary,"[AMAZON EARNINGS PREVIEW: ANOTHER LOSS, PRICE ...",475,HOLD
4,5,20180206_BTIG_AAPL_Apple-_Inc..pdf,2018-02-06,BTIG,AAPL,Apple Inc.,Technology,[(212) 527-3523 We are not changing our 2018 C...,198.00,BUY
5,6,20210820_Phillip_Securities_AAPL_Apple_Inc_Sup...,2021-08-20,Phillip Securities,AAPL,Apple Inc.,Technology,[Company Background Apple’s revenue comes from...,None,BUY
6,7,20220803_Phillip_Securities_AAPL_Apple_Inc_Man...,2022-08-03,Phillip Securities,AAPL,Apple Inc.,Technology,[3Q22 revenue and PATMI in line with expectati...,None,BUY
7,8,20150716_JP_Morgan_AAPL_Rod_Hall-s_Daily_Downl...,2015-07-16,JP Morgan,AAPL,Apple Inc.,Technology,[],None,Overweight
8,9,20171205_JP_Morgan_AMZN_Holiday_eComm_Update-_...,2017-12-05,JP Morgan,AMZN,Amazon.com Inc.,Consumer Discretionary,[1) Holiday online sales tracking ~17% thus fa...,None,None
9,10,20221123_Deutsche_Bank_WMT_Walmart_Inc-_3Q_Int...,2022-11-23,Deutsche Bank,WMT,Walmart Inc.,Consumer Staples,[WMT International drove solid results posting...,168.00,Company


# Get Date and Price for Dates

In [222]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            93 non-null     int64         
 1   filename      93 non-null     object        
 2   date          93 non-null     datetime64[ns]
 3   provider      93 non-null     object        
 4   ticker        93 non-null     object        
 5   company_name  93 non-null     object        
 6   industry      93 non-null     object        
 7   paragraphs    93 non-null     object        
 8   target_price  74 non-null     object        
 9   rating        87 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 7.4+ KB


In [158]:
"""# Apply DateOffset for each row using 'apply' on the DataFrame
df_reports["date next day"] = df_reports["date"].apply(lambda date: date + pd.DateOffset(days=1))
df_reports["date after 3 months"] = df_reports["date"].apply(lambda date: date + pd.DateOffset(months=3))
df_reports["date after 6 months"] = df_reports["date"].apply(lambda date: date + pd.DateOffset(months=6))
df_reports["date after 9 months"] = df_reports["date"].apply(lambda date: date + pd.DateOffset(months=9))
df_reports["date after 12 months"] = df_reports["date"].apply(lambda date: date + pd.DateOffset(months=12))"""

In [224]:
# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

In [225]:
df_saved_reports = df_reports

In [226]:
df_saved_reports

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-10-01,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,[BOTTOM LINE: We and almost every other invest...,"4,500",BUY
1,2,20190730_FinTrust_Investment_Advisors_AMZN_Fin...,2019-07-30,FinTrust Investment Advisors,AMZN,Amazon.com Inc.,Consumer Discretionary,[We are issuing this special report due to the...,"1,611",HOLD
2,3,20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...,2022-04-27,Wells Fargo,AAPL,Apple Inc.,Technology,"[Heading into Apple's F2Q22 earnings on 4/28, ...",205.00,Overweight
3,4,20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf,2015-07-28,BGC Partners,AMZN,Amazon.com Inc.,Consumer Discretionary,"[AMAZON EARNINGS PREVIEW: ANOTHER LOSS, PRICE ...",475,HOLD
4,5,20180206_BTIG_AAPL_Apple-_Inc..pdf,2018-02-06,BTIG,AAPL,Apple Inc.,Technology,[(212) 527-3523 We are not changing our 2018 C...,198.00,BUY
...,...,...,...,...,...,...,...,...,...,...
88,89,20220121_Wells_Fargo_JPM_JPM-_Downgrade_Rating...,2022-01-21,Wells Fargo,JPM,JPMorgan Chase & Co.,Financials,[We downgrade our rating from Overweight to Eq...,180.00,Change
89,90,20150804_BGC_Partners_AAPL_BGC_AAPL_07282015.pdf,2015-08-04,BGC Partners,AAPL,Apple Inc.,Technology,"[APPLE REVIEW: PREPARE FOR SLOWING GROWTH., PR...",115,HOLD
90,91,20210217_Barclays_PEP_PEP_4Q20_Earnings_Prep.pdf,2021-02-17,Barclays,PEP,PepsiCo Inc.,Consumer Staples,"[Heading into 4Q20 earnings, we expect 2021 gu...",147.00,Overweight
91,92,20191218_Barclays_JPM_JPM_CFO_Commentary_at_Se...,2019-12-18,Barclays,JPM,JPMorgan Chase & Co.,Financials,"[Earlier today, 7.5 months into her new role, ...",140.00,Overweight


In [227]:
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price), float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data), float(next_available_data)
            else:
                return float('nan'), float('nan')  # Return NaN for missing data

    except KeyError:
        return float('nan'), float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'), float('nan')  # Return NaN for missing data

def calculate_prices(row):
    short_name = row['ticker']
    base_date = pd.to_datetime(row['date'], utc=True)

    # Calculate prices
    row['start price'] = get_stock_prices(short_name, base_date)[0]
    row['one day after'] = get_stock_prices(short_name, base_date + pd.DateOffset(days=1))[0]
    row['max price after 3 months'], row["min price after 3 months"] = get_stock_prices(
        short_name, base_date, base_date + pd.DateOffset(months=3)
    )
    row['max price after 6 months'], row['min price after 6 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=3), base_date + pd.DateOffset(months=6)
    )
    row['max price after 9 months'], row['min price after 9 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=6), base_date + pd.DateOffset(months=9)
    )
    row['max price after 12 months'], row['min price after 12 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=9), base_date + pd.DateOffset(months=12)
    )
    return row

# Apply the function to each row
df_saved_reports = df_saved_reports.apply(calculate_prices, axis=1)


In [228]:
df_saved_reports.head()

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,start price,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months
0,1,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-10-01,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,[BOTTOM LINE: We and almost every other invest...,"4,500",BUY,161.063004,156.250000,172.181503,150.223999,169.000000,147.597504,175.272003,157.597000,186.570496,159.387497
1,2,20190730_FinTrust_Investment_Advisors_AMZN_Fin...,2019-07-30,FinTrust Investment Advisors,AMZN,Amazon.com Inc.,Consumer Discretionary,[We are issuing this special report due to the...,"1,611",HOLD,94.926498,93.338997,94.926498,85.275497,95.343002,86.735497,123.699997,83.830498,160.000000,114.302002
2,3,20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...,2022-04-27,Wells Fargo,AAPL,Apple Inc.,Technology,"[Heading into Apple's F2Q22 earnings on 4/28, ...",205.00,Overweight,154.242905,161.207794,163.552414,128.315125,172.447479,136.535339,153.864059,124.728371,128.577881,128.577881
3,4,20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf,2015-07-28,BGC Partners,AMZN,Amazon.com Inc.,Consumer Discretionary,"[AMAZON EARNINGS PREVIEW: ANOTHER LOSS, PRICE ...",475,HOLD,26.301500,26.450001,30.855000,23.168501,34.698502,28.509001,31.767500,24.103500,37.688999,30.100000
4,5,20180206_BTIG_AAPL_Apple-_Inc..pdf,2018-02-06,BTIG,AAPL,Apple Inc.,Technology,[(212) 527-3523 We are not changing our 2018 C...,198.00,BUY,38.352978,37.531948,43.422539,36.499203,49.574902,43.196339,55.221676,47.968876,49.958157,33.952549


In [229]:
df_saved_reports["target_price"] = (
    df_saved_reports["target_price"]
    .replace(",", "", regex=True)  # Remove commas
    .astype(float)                 # Convert to float
)

# Calculate performance of Target Prices

In [230]:
#If the target_price is below the current price, we need to check if it was below the target_price
def classifcy_performance(df):
    if df["target_price"] > df["start price"]:
        df["tp reached after 3 months"] = df["target_price"] <= df["max price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] <= df["max price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] <= df["max price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["max price after 12 months"]
    else:
        df["tp reached after 3 months"] = df["target_price"] >= df["min price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] >= df["min price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] >= df["min price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["min price after 12 months"]

    return df

df_saved_reports = df_saved_reports.apply(classifcy_performance, axis=1)

In [231]:
df_saved_reports.head(10)

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,...,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months,tp reached after 3 months,tp reached after 6 months,tp reached after 9 months,tp reached after 12 months
0,1,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-10-01,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,[BOTTOM LINE: We and almost every other invest...,4500.0,BUY,...,169.000000,147.597504,175.272003,157.597000,186.570496,159.387497,False,False,False,False
1,2,20190730_FinTrust_Investment_Advisors_AMZN_Fin...,2019-07-30,FinTrust Investment Advisors,AMZN,Amazon.com Inc.,Consumer Discretionary,[We are issuing this special report due to the...,1611.0,HOLD,...,95.343002,86.735497,123.699997,83.830498,160.000000,114.302002,False,False,False,False
2,3,20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...,2022-04-27,Wells Fargo,AAPL,Apple Inc.,Technology,"[Heading into Apple's F2Q22 earnings on 4/28, ...",205.0,Overweight,...,172.447479,136.535339,153.864059,124.728371,128.577881,128.577881,False,False,False,False
3,4,20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf,2015-07-28,BGC Partners,AMZN,Amazon.com Inc.,Consumer Discretionary,"[AMAZON EARNINGS PREVIEW: ANOTHER LOSS, PRICE ...",475.0,HOLD,...,34.698502,28.509001,31.767500,24.103500,37.688999,30.100000,False,False,False,False
4,5,20180206_BTIG_AAPL_Apple-_Inc..pdf,2018-02-06,BTIG,AAPL,Apple Inc.,Technology,[(212) 527-3523 We are not changing our 2018 C...,198.0,BUY,...,49.574902,43.196339,55.221676,47.968876,49.958157,33.952549,False,False,False,False
5,6,20210820_Phillip_Securities_AAPL_Apple_Inc_Sup...,2021-08-20,Phillip Securities,AAPL,Apple Inc.,Technology,[Company Background Apple’s revenue comes from...,NaN,BUY,...,179.076599,154.282730,176.300095,135.507355,172.447479,128.315125,False,False,False,False
6,7,20220803_Phillip_Securities_AAPL_Apple_Inc_Man...,2022-08-03,Phillip Securities,AAPL,Apple Inc.,Technology,[3Q22 revenue and PATMI in line with expectati...,NaN,BUY,...,149.715576,124.728371,128.577881,128.577881,128.577881,128.577881,False,False,False,False
7,8,20150716_JP_Morgan_AAPL_Rod_Hall-s_Daily_Downl...,2015-07-16,JP Morgan,AAPL,Apple Inc.,Technology,[],NaN,Overweight,...,27.642393,21.844837,25.527149,21.158571,24.475096,20.697264,False,False,False,False
8,9,20171205_JP_Morgan_AMZN_Holiday_eComm_Update-_...,2017-12-05,JP Morgan,AMZN,Amazon.com Inc.,Consumer Discretionary,[1) Holiday online sales tracking ~17% thus fa...,NaN,None,...,84.817497,68.599503,101.975502,83.025497,100.649002,74.773003,False,False,False,False
9,10,20221123_Deutsche_Bank_WMT_Walmart_Inc-_3Q_Int...,2022-11-23,Deutsche Bank,WMT,Walmart Inc.,Consumer Staples,[WMT International drove solid results posting...,168.0,Company,...,46.103802,46.103802,46.103802,46.103802,46.103802,46.103802,False,False,False,False


In [249]:
df_testing = df_saved_reports

# Clean/Filter Paragraphs

In [254]:
import re

def clean_paragraph(paragraph):
    # Remove email addresses
    paragraph = re.sub(r"\S+@\S+", "", paragraph)
    
    # Remove phone numbers
    paragraph = re.sub(r"\b\d{1,3}[-.\s]?\d{3}[-.\s]?\d{4}\b", "", paragraph)
    
    # Remove URLs
    paragraph = re.sub(r"http\S+|www\S+", "", paragraph)
    
    # Remove special characters (keep alphanumerics, spaces, and common punctuation)
    paragraph = re.sub(r"[^\w\s,.!?]", "", paragraph)
    
    # Remove multiple spaces or newlines
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    
    # Remove boilerplate phrases
    boilerplate_phrases = ["Disclaimer", "Confidential", "For internal use only"]
    for phrase in boilerplate_phrases:
        paragraph = paragraph.replace(phrase, "")
    
    # Remove short sentences (fewer than 10 words)
    if len(paragraph.split()) < 10:
        return None
    
    return paragraph

# Apply cleaning function
testing_df['paragraphs'] = testing_df['paragraphs'].apply(
    lambda paragraphs: [clean_paragraph(p) for p in paragraphs if clean_paragraph(p)]
)

# Drop rows where the `paragraphs` column is empty after cleaning
testing_df = testing_df[testing_df['paragraphs'].str.len() > 0]

In [255]:
# Explode the cleaned list into separate rows
df_exploded = testing_df.explode('paragraphs').reset_index(drop=True)

In [256]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4921 entries, 0 to 4920
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ID                          4921 non-null   int64         
 1   filename                    4921 non-null   object        
 2   date                        4921 non-null   datetime64[ns]
 3   provider                    4921 non-null   object        
 4   ticker                      4921 non-null   object        
 5   company_name                4921 non-null   object        
 6   industry                    4921 non-null   object        
 7   paragraphs                  4921 non-null   object        
 8   target_price                4763 non-null   float64       
 9   rating                      4864 non-null   object        
 10  start price                 4921 non-null   float64       
 11  one day after               4921 non-null   float64     

In [87]:
#df_reports.to_csv("paragraphs.csv")